In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [83]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from textblob import TextBlob
from wordcloud import WordCloud

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        



In [84]:
train = pd.read_csv('/kaggle/input/south-african-language-identification-2021/train_set.csv')
test = pd.read_csv('/kaggle/input/south-african-language-identification-2021/test_set.csv')
sample = pd.read_csv('/kaggle/input/south-african-language-identification-2021/sample_submission.csv')
df = pd.concat([train, test], axis=0, sort = True)

In [85]:
train.head()

In [86]:
test.head()

In [87]:
sample.head()

In [88]:
train.lang_id.value_counts()

In [89]:
plot = train['lang_id'].value_counts()
sns.barplot(plot.index,plot)

In [90]:
df.head()

In [91]:
df.describe()

In [92]:
df.isnull().sum()

In [93]:
df1 = df.copy()

In [94]:
# df1['length'] = df1['text'].str.len() 
# maxlength = dfc['length'].max() 
# minlength = dfc['length'].min() 
# maxlength,minlength

In [95]:
cat = df1['lang_id'].unique()

In [96]:
# replace string to multiclass numeric
df1['lang_id'].replace({'xho':1,'eng':2,'nso':3,'ven':4, 'tsn':5, 'nbl':6, 'zul':7, 'ssw':8, 'tso':9,'sot':10, 'afr':11}, inplace=True)


In [97]:
df1.info()

In [98]:
df1.head(10)

In [99]:
lang_test.shape

In [100]:
df2 = pd.concat([lang_test.assign(ind="test"), lang_train.assign(ind="train")])

In [101]:
df2.head()

In [102]:
df2.shape

In [103]:
df2['text'] = df2['text'].str.lower()

In [104]:
df2.head()

In [105]:
#English
eng_stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

#Setswana
tsn_stopwords = ['tsa me','tsa','re','rona','tsa gago','gago','rraalona','èna','lona','dira','se','lo','se batlang','batlang','ga bōnè','ga',
'bōnè','fa','tsamaya','go']

#Sesotho
sot_stopwords = ['ke','nna','ka bonna','ka nna','ka','rona','ea rona','tsa rona','rona ka borona','uena','u joalo','joalo','u tla','tla','u ka','ea hau','uena',
'lona ka bolona','eena','hae','ka boeena','eona','ke eona','bona','tsa bona','tsa','ba','ena','ba le','le','e ne e','o ile a','hore','bakeng sa',
'e le','empa','eng','ba bang ba','kapa','ne','ea','ho','re','e leng','etsa','ba bona ba','haeba','joang','mong le e mong','etsang','mona','lokela ho',
'joalo','ke hobane’ng ha','theoha','lokela','re','hape','leha e le efe','ka mor’a hore','feela','mong le e mong','’na','rōna','tlas’a','haholo','ka ho','feela',
'haholo','pele','le eena o','tšoanang','o ile a','tsohle','mono','neng','hao','ka nako eo','e le','tsena','mo','o na le','hape','ka se khonang','ile a etsa',
'ha ho','ka ho fetisisa','ho feta','ba ba neng ba','bile','hona joale','lokela','pakeng tsa','se ke oa','ha a ntse a','tse seng kae','hang','ka bobeli','ho fihlela',
'ka nako ya','khahlanong le','etsa','ha e etse joalo','ha a so','ka a etsa joalo','mohlomong ha ho joalo','mohlomong','ha ho','ha ea lokela','ha e hloke','ha ho joalo',
 'ka mora','pele ho','nakong','kena','mabapi le','bakeng sa','nakong','ha a ntse a','ho fihlela',
 'kapa','tse ling','enngwe','hape','mmalwa',"ka 'ngoe",'ka bobeli','ho tswela pele',"tlas'a",'fetile','theoha','kantle','ka hare','tlase','nyoloha','ho tloha','ka tlase',
'che','tsa hao','hape','se ke oa','lokela ho ba le','ne a ke ke','entse','ha aa ka a etsa joalo' ,'ke ke','ha ea lokela','e ne e se','li ne li se joalo','ha ho joalo','a','ba','bane','bona','e','ea','eaba','empa','ena','ha','hae','hape','ho','hore','ka','ke','la','le','li','me','mo','moo','ne','o','oa','re','sa','se','tloha','tsa','tse','eba','aowa','mohlômong','mohlômongwê','tše','tša','go']                 

#Xitsonga
tso_stopwords = ['xewani','avuxeni','ahee','kunjhani','minjhani','ni','kona','ndzi','ndza','vonani','hi','wena','ra','huma','e','kwihi','mi',
'ku','u','va','ina','e-e','kumbe','xana','a','tivi','kombela','vula','sweswo','nakambe','xi','xana','wa','ke','mina','ndza','ri',
'ku']

#Afrikaans
afr_stopwords = ['ek','my','ek self','myself','ons',"ons s'n",'onsself','jy','jou','julle','jy het','het','jy sal','sal','jy sou','sou','joune','jouself',
'julleself','hy','hom','syne','sy','homself','sy is','is','haar','haarne','haarself','dit','dit is','self','hulle','hul',"hulle s'n",
'hulself','wat','watter','wie','hierdie','daardie','dit sal','die','was','wees','gewees','wese','het','hê','gehad het','gehad','doen','het gedoen',
'en','maar','as','of','omdat','tot','totdat','terwyl','wyle','van','by','teen','deur','met','vir','oor','om', 'omheen', 'rondon','tussen','in','gedurende',
'voor','daarna','na','hierbo','bo','hieronder','onder','aan','tot','van','up','af','in','uit','buite','op','verby', 'weer','verder','dan','destyds','destyd',
'een keer','een maal','hier','daar','wanneer','waar','hoekom','hoe','almal','alle','enige','beide','elk','elke','min','meer','die meeste','meeste','ander',
'sommige','sulke','sodanig','geen','nee','ook nie','nie','ook','enigste','net','slegs','sleg','eie','besit','dieselfde','so','as','alte','baie','kan','blik',
'moenie','moes','nou','en','is nie','kon nie','kon','het nie','doen nie','doen','moes nie','mag nie','mag','moenie','moet','nodig het','het nie nodig nie','moet nie',
'was nie','was','sou nie','sou']

#isiXhosa
xho_stopwords = ['mna','wam','ngokwam','thina','yethu','eyethu','wena','nguwe','une','uza','ungathanda','eyakho','ngokwakho','ngokwenu','nina',
'yena','yakhe','ngokwakhe','eyakhe','yiyo','ngokwayo','bona','kubo','yabo','eyabo','ngeyabo','ngokwabo','intoni','ntoni','eyiphi','ngeyiphi',
'ngubani','ungubani','engubani','le','leya','lonto','izakuza','ezi','eziya','ezo','ndim','ngu','wayenjalo','unjalo','babenjalo','njalo',
'kuba','ukuba','ngokuba','sele','unayo','nayo','ebe','unayo','ndinayo','be','a','e','i','o','u','ukuba','yenza','uzoyenza','zoyenza','ukwenza',
'uyenzile','wenzile','ukwenzile','kwenzile','kwenza','ukwenza','kwaye','okunye','kunye','kodwa','ukuba','kuba','okanye','kanye','kuba','ukuba','ngoba',
'njenge','kude kube','kude','kube','ngeli xesha','ngeli','xesha','xana','xa','na','ye','nge','nje','ngokuba','malunga','ngokuchasene','chasene','bachasene','chasile','umchasile','aka','kaloku',
'phakathi','ungene','ukugqitha','gqitha','gqithile','gqithisa','gqithisile','ngexesha','ngaphambili','emva','mva','ngentla','ntla',
'ngezantsi','zantsi','ukuya','kuya','ukusuka','kusuka','suka','phezulu','ngaphezulu','nangaphezulu','na ngaphezulu','phantsi',
'ngaphantsi','nga phantsi','phakathi','ngaphandle','phandle','ivuliwe','vuliwe','icimile','cimile','ngaphaya','phaya',
'kwakhona','khona','kwa','kamva','emva koko','koko','emveni','apha','phaya','ya','pha','ngaphaya','ingathi','ngathi','nini','nithini',
 'uthini','thini','bathini','bathi','athi','besithi','esithi','sithi','phi','njani','unjani','onjani','banjani','konke','bonke','nayiphi na','nayiphi',
'zombini','nganye','zimbalwa','ezimbalwa','kaninzi','ninzi','nintsi','kanintsi','ezinintsi','inintsi','uninzi','isininzi','sininzi',
'enye','ezinye','enjalo','injalo','kunjalo','njalo','hayi','kuphela','yeyakho','nge yakho','yakho','nge','ngokufanayo','okufanayo','oku','ofanayo','fanayo',
'ke','kunokuba','nokuba','kuba','naye','ebe','kakhulu','khulu','ka','unako','ubunako','ebenako','ubu nako','ebe nako','ngaba',
'nje','andazi','andiyazi','ndiyayazi','ndiyazi','kufanele','okufaneleyo','kufanelekile','ngoku','ngokuya','kwakhona','kwaye',
'ngaba','akunjalo','kunjalo','ayikwazi','ayenzi','eyenza','enza','khange','ayisiyiyo','ayiseyiyo','íyiyo','yiyo','le','ngekhe',
'akufuneki','kufuneka','wayengekho','ebengekho','ngekho','ebekho','ukho','babengekho','bengekho','angekhe','soze','mhlawumbi','eli','elo','nelo','neli','nalu','eliya','eliyana']

#isiZulu
zul_stopwords = ["futhi","kahle","kakhulu","kanye","khona","kodwa","kungani","kusho","la","lakhe","lapho","mina","ngesikhathi","nje","phansi","phezulu","u","ukuba","ukuthi","ukuze","uma","wahamba","wakhe","wami","wase","wathi","yakhe","zakhe","zonke",
'mina','mina','wami','thina','yethu','okwethu','wena','uwena','uke','uzoba','ubungeke','ngeke','eyakho','wenu','nina','yena','okwakhe','unjalo','kanjalo','njalo','kunjalo','eyakhe','yena','kuyinto','uqobo','qobo','bona','bona','yona','zona',
 'yabo','ezabo','zabo','ngokwabo','lwabo','lwazo','yini','kuyini','ngani','manje','khona','lapho','okuyi','imuphi','muphi', 'ubani','bani','ngubani','lokhu','lokhuya','lesi','leli','ukuthi','lokho kuzoba','lokho','kuzoba','lezi','labo','lezo','lawo',
'ngi','yimi','kuyinto','kukhona','kwaba','zazikhona','zikhona','ikhona','kube','babe','bekulokhu','ukuba','unayo','banayo','kwadingeka','dingeka','kokuba','yenza','kuyakwenza','wenze','ukwenza','futhi','nafuthi','kanye','nakanye','kodwa','kepha','kanti',
'uma','nxa','noma','kumbe','kambe','konje','yini','manje','ngoba','ngalokho','lokho','njengoba','kuze kube','kuze kwabe','kuze','kwabe','ngenkathi','ngesikhathi','lapho','ye','ngase','duze','ngoba','nge','ne','na','mayelana','maqondana','ngokumelene','ncikene',
'ncikene','ebhekene','nqikene','phakathi','ngaphakathi','ungene','kungene','bangene','ngokusebenzisa','ngesikhathi','mhlana','mhla','kusa','ngaphambi','ngaphambili','phambili','ngemuva','emuva','kamuva','ngenhla','enhla','ngezansi','phansi','ngaphansi','ezansi',
'uku','kusuka','phezulu','ngaphezulu','enhla','phakathi','kuphakathi','kwaphakathi','phuma','phandle','vula','kuvaliwe','valiwe','cishe','cishile','cishiwe','ungene','phezu','seleyo','eqela','futhi','buye','phinda','waphinda','ngaphinda','ngokuqhubekayo','phambili',
'lapho-ke','lapho ke','lapho','ke','kanye','lapha','la','laphaya','nini','manxa','ngesikhathi','kuphi','phi','ngani','ngobani','ubani','ngani','yini','kanjani','njani','unjani','konke','kuze','noma yini','noma kuphi','kokubili','bobabili','yombili','ngamunye','emunye',
'sisinye','lowo nalowo','lowo','nalowo','okumbalwa','mbalwa','iningana','ncane','okuningi','kuningi','ngokweqileyo','ningi ngokwedlulele','ngokwedlulele','kakhulu','okunye','ezinye','nye','yingcosana','enjalo','njalo','ngenga','nje','cha','qha','ke','noma','hhayi','kuphela',
'okwakho','wami','lami','yami','ngeyami','nge yami','ngowami','ngelami','ngo wami','nge lami','ngokufanayo','ngo kufanayo','fanayo','ngakho-ke','ngakho ke','ngakho','ke','kune','kunoba','kunokuthi','kuna','futhi','ngokweqile','kakhulu','impela','ngempela','nge mpela','-noku','noku',
'ngakwazi','nga kwazi','kuthanda','isifiso','intando','nje','ungakwenzi','kufanele','ngokufanele','ngo kufanele','bekufanele','beku','manje','kabusha','akuzona','kuzona','zona','ayikwazanga','ayikwazanga','akazange','akusho','kusho','akunjalo','kunjalo','kanjalo','njalo','ngangingenalo',
'nganginalo','nginalo','akukaze','kaze','angikaze','akuyona','kuyona','kungenzeka','kungenzeki','akumele','kumele','akudingeki','isidingo','ukudinga','ukufuna','funa','dinga','akunjalo','akufanele','kwakungeyona','yona','babengekho','ngeke']

#Tshivenda
ven_stopwords = ['nne','inwi','ene','ula','vhala','fhala','tshila','tshone','ndi','la','ya','ni','vhone',' khou','na','nnyi','bva','ga',' fhi',
 'gai','do','lini','zwavhudi','livhuwa','khonani','pfi','vhona','vuwa','nga','si','kone','ha','a','naa','ee!','ee','hai!','hai','futhi','habe',
'tou','thi','u','ngafhi','ndo','ri','rine','vha','inwi','wa','la','tsha','zwa','dza','la','mini','ngani','hani','mini','-ifhio','ifhio',
'aa','ndaa','ro','pfa','khamusi','zwo','fara','duvha','li','divhi','humbela','vhe','vho','dzi','dze','dzo','dzu','yo','yi','yu','ye','yu']

stopwords_all = ven_stopwords + zul_stopwords + xho_stopwords + afr_stopwords + tso_stopwords + sot_stopwords + tsn_stopwords + eng_stopwords

filter_char = lambda c: ord(c) < 256

In [106]:
df2['text'] = df2['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords_all]))

In [107]:
df2['text']= df2['text'].apply(lambda s: ''.join(filter(filter_char, s)))

In [109]:
lang_train = train.copy()
lang_test = test.copy()

In [110]:
lang_train.shape


In [111]:
lang_test.shape

In [112]:
DF = pd.concat([lang_test.assign(ind="test"), lang_train.assign(ind="train")])

In [113]:
DF['text'] = DF['text'].str.lower()

In [114]:
DF.head()

In [115]:
tdif = TfidfVectorizer(strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',stop_words=None, ngram_range=(1,1))

In [116]:
X_DF = tdif.fit_transform(DF['text'])
X_test = X_DF[0:5682]
X_train = X_DF[5682:]
y = lang_train['lang_id']

In [117]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y)

In [118]:
rfc_pred = rfc.predict(X_test)
pred_lang = pd.DataFrame(rfc_pred,columns=['lang_id'])
submission = pd.DataFrame({'index':lang_test.index + 1,'lang_id':pred_lang.lang_id})

In [119]:
submission.to_csv('hackathonC2022.csv',index=False)
